# SW-4b-Python-SPARQL

**Sidetrack Python** : [<< SW-4-CSharp-SPARQL](SW-4-CSharp-SPARQL.ipynb) | [Index](README.md) | [SW-5-CSharp-LinkedData >>](SW-5-CSharp-LinkedData.ipynb)

## SPARQL en Python avec rdflib

Ce notebook est un **sidetrack optionnel** qui presente l'equivalent Python des concepts SPARQL du notebook SW-4 (dotNetRDF). Vous y decouvrirez comment executer des requetes SPARQL sur des graphes en memoire avec **rdflib**.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Executer des requetes SPARQL SELECT sur des graphes rdflib
2. Utiliser FILTER, OPTIONAL, UNION et ORDER BY
3. Interroger des hierarchies de classes RDFS
4. Faire la correspondance entre dotNetRDF et rdflib pour SPARQL

### Prerequis
- SW-4-CSharp-SPARQL recommande (pour la comprehension conceptuelle)
- Python 3.10+

### Duree estimee : 25 minutes

> **Note** : Ce notebook se concentre sur SPARQL local. Pour les endpoints distants (DBpedia, Wikidata), voir le sidetrack **SW-5b-Python-LinkedData**.

---

## 1. Installation et Preparation

rdflib integre un moteur SPARQL complet qui permet d'interroger les graphes en memoire.

In [1]:
%pip install -q rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from rdflib import Graph, Namespace

# Load animals.ttl - a graph with RDFS class hierarchy
g = Graph()
g.parse("data/animals.ttl", format="turtle")

print(f"Triples charges depuis animals.ttl : {len(g)}")
print()

# Show namespaces
print("Namespaces utilises :")
for prefix, uri in g.namespaces():
    if prefix and prefix in ['ex', 'rdfs', 'rdf']:
        print(f"  @prefix {prefix}: <{uri}> .")

Triples charges depuis animals.ttl : 51

Namespaces utilises :
  @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
  @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
  @prefix ex: <http://example.org/animals#> .


### Interpretation

Le fichier `animals.ttl` contient :
- Une hierarchie de classes : `Animal > Mammal/Bird > Dog/Cat/Parrot`
- Des instances avec proprietes : `name`, `age`, `sound`, `canFly`

C'est le meme fichier utilise dans SW-4 (dotNetRDF), ce qui nous permet de comparer directement les syntaxes.

---

## 2. Requete SELECT Simple

Commencons par lister tous les animaux avec leur nom et leur cri.

In [3]:
# Simple SELECT query - list all animals with name and sound
query = """
PREFIX ex: <http://example.org/animals#>

SELECT ?animal ?name ?sound
WHERE {
    ?animal ex:name ?name .
    ?animal ex:sound ?sound .
}
ORDER BY ?name
"""

results = g.query(query)

print(f"Resultats : {len(results)} animaux trouves")
print(f"{'Animal':<35s} {'Nom':<12s} {'Cri':<25s}")
print("-" * 72)
for row in results:
    print(f"{str(row.animal):<35s} {str(row.name):<12s} {str(row.sound):<25s}")

Resultats : 4 animaux trouves
Animal                              Nom          Cri                      
------------------------------------------------------------------------
http://example.org/animals#buddy    Buddy        Woof woof                
http://example.org/animals#coco     Coco         Coco veut un gateau      
http://example.org/animals#minou    Minou        Miaou                    
http://example.org/animals#rex      Rex          Woof                     


### Interpretation : Requete SELECT

La requete retourne les 4 animaux definis dans `animals.ttl`.

| Element | Syntaxe SPARQL | Description |
|---------|---------------|-------------|
| PREFIX | `PREFIX ex: <...>` | Declare un prefixe (equivalent de `@prefix` en Turtle) |
| Variables | `?animal`, `?name` | Variables SPARQL (commencent par `?` ou `$`) |
| Pattern | `?animal ex:name ?name .` | Triple pattern a matcher |
| ORDER BY | `ORDER BY ?name` | Tri ascendant |

**rdflib vs dotNetRDF** :
| Operation | rdflib | dotNetRDF |
|-----------|---------|----------|
| Executer requete | `g.query("SELECT...")` | `g.ExecuteQuery("SELECT...")` |
| Type de retour | `Result` (iterable) | `SparqlResultSet` |
| Acces valeur | `row.name` ou `row[1]` | `row["name"]` |
| Compter resultats | `len(results)` | `results.Count` |

---

## 3. Filtrage avec FILTER

Le mot-cle `FILTER` applique des conditions aux resultats.

In [4]:
# FILTER: find animals older than 4 years
query_filter = """
PREFIX ex: <http://example.org/animals#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?name ?age
WHERE {
    ?animal ex:name ?name .
    ?animal ex:age ?age .
    FILTER (?age > 4)
}
ORDER BY DESC(?age)
"""

print("=== Animaux de plus de 4 ans ===")
for row in g.query(query_filter):
    print(f"  {row.name} : {row.age} ans")

=== Animaux de plus de 4 ans ===
  Coco : 12 ans
  Buddy : 7 ans
  Rex : 5 ans


### Interpretation : FILTER

Le `FILTER (?age > 4)` ne retient que les animaux dont l'age depasse 4 ans.

| Operateur | Description | Exemple |
|-----------|-------------|----------|
| Comparaisons | `=`, `!=`, `<`, `>`, `<=`, `>=` | `FILTER (?age > 4)` |
| Logique | `&&`, `||`, `!` | `FILTER (?age > 4 && ?age < 10)` |
| Tests de type | `isURI()`, `isBlank()`, `isLiteral()` | `FILTER (isURI(?x))` |
| Regex | `REGEX()` | `FILTER (REGEX(?name, "^C"))` |
| Existence | `BOUND()` | `FILTER (BOUND(?optional))` |

Equivalent dotNetRDF : Meme syntaxe SPARQL, mais `SparqlParameterizedString` pour les requetes parametrees.

---

## 4. Joins Optionnels avec OPTIONAL

Le mot-cle `OPTIONAL` inclut un pattern meme s'il n'est pas satisfait (equivalent de LEFT JOIN en SQL).

In [5]:
# OPTIONAL: list all animals with flight capability (if known)
query_optional = """
PREFIX ex: <http://example.org/animals#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?type_label ?canFly
WHERE {
    ?animal ex:name ?name .
    ?animal a ?type .
    ?type rdfs:label ?type_label .
    OPTIONAL { ?animal ex:canFly ?canFly . }
}
ORDER BY ?name
"""

print("=== Animaux avec capacite de vol (OPTIONAL) ===")
print(f"{'Nom':<12s} {'Type':<15s} {'Peut voler':<12s}")
print("-" * 39)
for row in g.query(query_optional):
    fly_str = str(row.canFly) if row.canFly is not None else "non renseigne"
    print(f"{str(row.name):<12s} {str(row.type_label):<15s} {fly_str:<12s}")

=== Animaux avec capacite de vol (OPTIONAL) ===
Nom          Type            Peut voler  
---------------------------------------
Buddy        Chien           non renseigne
Coco         Perroquet       true        
Minou        Chat            non renseigne
Rex          Chien           non renseigne


### Interpretation : OPTIONAL

Seul Coco (perroquet) a la propriete `ex:canFly`. Grace a `OPTIONAL`, les autres animaux apparaissent quand meme dans les resultats.

| Mot-cle SPARQL | Equivalent SQL | Comportement |
|---------------|----------------|-------------|
| `FILTER` | `WHERE condition` | Exclut les lignes ne respectant pas la condition |
| `OPTIONAL` | `LEFT JOIN` | Inclut la ligne meme si le pattern optionnel n'est pas satisfait |
| `ORDER BY` | `ORDER BY` | Tri ascendant (defaut) ou `DESC()` |

> **Principe du monde ouvert** : L'absence d'information ne signifie pas que l'information est fausse. `OPTIONAL` respecte ce principe.

---

## 5. Combinaison de Patterns avec UNION

Le mot-cle `UNION` combine les resultats de plusieurs patterns (equivalent de `UNION ALL` en SQL).

In [6]:
# UNION: mammals OR birds
query_union = """
PREFIX ex: <http://example.org/animals#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?type_label
WHERE {
    ?animal ex:name ?name .
    {
        ?animal a ex:Mammal .
        ex:Mammal rdfs:label ?type_label .
    } UNION {
        ?animal a ex:Bird .
        ex:Bird rdfs:label ?type_label .
    }
}
ORDER BY ?name
"""

print("=== Mammiferes OU Oiseaux ===")
for row in g.query(query_union):
    print(f"  {row.name} - {row.type_label}")

=== Mammiferes OU Oiseaux ===


### Interpretation : UNION

`UNION` combine les resultats de deux patterns. C'est utile quand une propriete peut etre exprimee de plusieurs facons differentes.

Equivalent dotNetRDF : La syntaxe SPARQL est identique, seul le wrapping change.

---

## 6. Exploration de la Hierarchie de Classes

Le fichier `animals.ttl` contient une hierarchie RDFS. Interrogeons-la pour comprendre les relations entre classes.

In [7]:
# Query class hierarchy
query_classes = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ex: <http://example.org/animals#>

SELECT ?class ?label ?parent ?parent_label
WHERE {
    VALUES ?class { ex:Animal ex:Mammal ex:Bird ex:Dog ex:Cat ex:Parrot }
    ?class rdfs:label ?label .
    OPTIONAL {
        ?class rdfs:subClassOf ?parent .
        ?parent rdfs:label ?parent_label .
    }
}
ORDER BY ?parent_label ?label
"""

print("=== Hierarchie de classes ===")
print(f"{'Classe':<15s} {'Parent':<15s}")
print("-" * 30)
for row in g.query(query_classes):
    parent = str(row.parent_label) if row.parent_label else "(racine)"
    print(f"{str(row.label):<15s} {parent:<15s}")

=== Hierarchie de classes ===
Classe          Parent         
------------------------------
Animal          (racine)       
Mammifere       Animal         
Oiseau          Animal         
Chat            Mammifere      
Chien           Mammifere      
Perroquet       Oiseau         


### Interpretation : Hierarchie de classes

La hierarchie est visible : `Animal` est la racine, `Mammal` et `Bird` sont des sous-classes directes.

| Propriete RDFS | Description |
|----------------|-------------|
| `rdfs:subClassOf` | Hierarchie de classes (transitive) |
| `rdfs:label` | Nom lisible pour une ressource |
| `rdf:type` / `a` | Relation instance-classe |

---

## 7. Pagination avec LIMIT et OFFSET

Pour les gros resultats, `LIMIT` et `OFFSET` permettent la pagination.

In [8]:
# Pagination: first 2 animals, then next 2
query_page1 = """
PREFIX ex: <http://example.org/animals#>

SELECT ?name ?age
WHERE {
    ?animal ex:name ?name .
    ?animal ex:age ?age .
}
ORDER BY ?name
LIMIT 2
"""

query_page2 = """
PREFIX ex: <http://example.org/animals#>

SELECT ?name ?age
WHERE {
    ?animal ex:name ?name .
    ?animal ex:age ?age .
}
ORDER BY ?name
LIMIT 2
OFFSET 2
"""

print("=== Page 1 (LIMIT 2) ===")
for row in g.query(query_page1):
    print(f"  {row.name} : {row.age} ans")
print()
print("=== Page 2 (LIMIT 2 OFFSET 2) ===")
for row in g.query(query_page2):
    print(f"  {row.name} : {row.age} ans")

=== Page 1 (LIMIT 2) ===
  Buddy : 7 ans
  Coco : 12 ans

=== Page 2 (LIMIT 2 OFFSET 2) ===
  Minou : 3 ans
  Rex : 5 ans


### Interpretation : Pagination

- `LIMIT 2` : Retourne au maximum 2 resultats
- `OFFSET 2` : Saute les 2 premiers resultats

C'est essentiel pour les endpoints publics qui peuvent avoir des millions de triples.

---

## 8. Tableau de Correspondance dotNetRDF / rdflib (SPARQL)

Ce tableau recapitule les equivalences pour les requetes SPARQL.

| Operation | dotNetRDF (C#) | rdflib (Python) |
|-----------|---------------|----------------|
| **Executer SELECT** | `g.ExecuteQuery("SELECT...")` | `g.query("SELECT...")` |
| **Type de retour** | `SparqlResultSet` | `Result` (iterable) |
| **Iterer resultats** | `foreach (SparqlResult row in results)` | `for row in results:` |
| **Acces valeur** | `row["name"]` | `row.name` ou `row[0]` |
| **Compter** | `results.Count` | `len(results)` |
| **Requete parametree** | `SparqlParameterizedString` | f-string ou format Python |
| **Endpoint distant** | `SparqlRemoteEndpoint` | `SPARQLWrapper` (voir SW-5b) |

### Syntaxe SPARQL - Identique dans les deux bibliotheques

La syntaxe SPARQL elle-meme est **identique** - seuls changent les wrappers d'execution :

```sparql
PREFIX ex: <http://example.org/>

SELECT ?s ?p ?o
WHERE {
    ?s ?p ?o .
    FILTER (?o > 10)
    OPTIONAL { ?s ex:optional ?opt . }
}
ORDER BY ?s
LIMIT 10
```

---

## Exercices

Mettez en pratique les concepts SPARQL appris.

### Exercice 1 : Trouver les mammiferes

Ecrivez une requete SPARQL qui retourne tous les animaux qui sont des mammiferes (instances de `ex:Mammal` ou de ses sous-classes `ex:Dog`, `ex:Cat`), avec leur nom et leur age.

**Indice** : Utilisez `rdfs:subClassOf` pour remonter la hierarchie, ou listez les classes directement avec `UNION`.

In [9]:
# Exercice 1 : Trouver tous les mammiferes dans animals.ttl

query_mammals = """
PREFIX ex: <http://example.org/animals#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?age ?type_label
WHERE {
    # TODO: Completez la requete
    # Trouvez les animaux dont le type est une sous-classe de ex:Mammal
}
ORDER BY ?name
"""

# for row in g.query(query_mammals):
#     print(f"{row.name} ({row.type_label}) - {row.age} ans")

### Exercice 2 : Animaux sans age defini

Ecrivez une requete qui trouve tous les animaux qui n'ont PAS de propriete `ex:age` definie.

**Indice** : Utilisez `FILTER (!BOUND(?age))` ou le negatif pattern `MINUS`.

In [10]:
# Exercice 2 : Animaux sans age defini

query_no_age = """
PREFIX ex: <http://example.org/animals#>

SELECT ?name
WHERE {
    # TODO: Completez la requete
    # Trouvez les animaux sans age
}
"""

# for row in g.query(query_no_age):
#     print(f"  {row.name} n'a pas d'age defini")

---

## Resume

Ce sidetrack a presente l'execution de requetes SPARQL en Python avec **rdflib**.

### Concepts cles

| Concept | Ce que vous avez appris |
|---------|------------------------|
| **SELECT** | Projeter des variables depuis un graphe |
| **FILTER** | Appliquer des conditions aux resultats |
| **OPTIONAL** | Jointures externes (valeurs manquantes autorisees) |
| **UNION** | Combiner plusieurs patterns |
| **ORDER BY / LIMIT / OFFSET** | Trier et paginer les resultats |
| **Correspondance** | Equivalences dotNetRDF / rdflib |

### Prochaines etapes

- **SW-5-CSharp-LinkedData** : Donnees liees en .NET (DBpedia, Wikidata)
- **SW-5b-Python-LinkedData** : Interroger des endpoints distants avec SPARQLWrapper
- **SW-6-CSharp-RDFS** : Schema et inference en .NET

---

**Navigation** : [<< SW-4-CSharp-SPARQL](SW-4-CSharp-SPARQL.ipynb) | [Index](README.md) | [SW-5-CSharp-LinkedData >>](SW-5-CSharp-LinkedData.ipynb)